In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import pandas as pd
import random
import itertools
import IPython.display as ipd
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers


%matplotlib inline

In [ ]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
import os
from os import listdir
from os.path import isfile, join

In [ ]:
#DataFlair - Emotions in the CREAMA-D dataset
emotions={
  'ANG':'angry',
  'FEA':'fear',
  'HAP':'happy',
  'NEU':'neutral',
  'SAD':'sad',
  'DIS':'disgust'
}

#DataFlair - Emotions to observe
observed_emotions=['angry', 'fear', 'happy', 'neutral', 'sad']

In [ ]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def wn_extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        data = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        wn = np.random.randn(len(data))
        X = data + 0.005*wn
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def shift_extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        data = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        X = np.roll(data, 1600)
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def stretch_extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        data = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        X = librosa.effects.time_stretch(data, 1.6)
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:
## Dataset to be predicted

images = {}
im2 = []

for root, dirs, files in os.walk("/content/drive/MyDrive/Qualifiers/NLP Interim Dataset/NLP"):
   path = root.split(os.sep)
   for index, file in enumerate(files):
      im2 = [ f for f in listdir(root) if isfile(join(root,f)) ]
      images[index] = join(root,im2[index])

In [ ]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.01):
    x,y=[],[]
    
    #training set
    for file in glob.glob("/content/drive/MyDrive/Qualifiers/NLP Training Dataset/ASR Training Dataset/angry/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("angry")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("angry")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("angry")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("angry")

    for file in glob.glob("/content/drive/MyDrive/Qualifiers/NLP Training Dataset/ASR Training Dataset/fear/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("fear")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("fear")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("fear")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("fear")

    for file in glob.glob("/content/drive/MyDrive/Qualifiers/NLP Training Dataset/ASR Training Dataset/happy/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("happy")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("happy")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("happy")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("happy")

    for file in glob.glob("/content/drive/MyDrive/Qualifiers/NLP Training Dataset/ASR Training Dataset/neutral/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("neutral")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("neutral")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("neutral")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("neutral")

    for file in glob.glob("/content/drive/MyDrive/Qualifiers/NLP Training Dataset/ASR Training Dataset/sad/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("sad")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("sad")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("sad")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("sad")
    
    
    # 4 June Easter Egg     
    for file in glob.glob("/content/drive/MyDrive/NLP easter 4 june/angry/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("angry")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("angry")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("angry")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("angry")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 4 june/fear/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("fear")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("fear")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("fear")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("fear")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 4 june/happy/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("happy")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("happy")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("happy")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("happy")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 4 june/neutral/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("neutral")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("neutral")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("neutral")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("neutral")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 4 june/sad/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("sad")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("sad")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("sad")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("sad")
  
    
    
    # 1 June Easter Egg
    for file in glob.glob("/content/drive/MyDrive/NLP easter 1 june/angry/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("angry")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("angry")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("angry")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("angry")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 1 june/fear/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("fear")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("fear")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("fear")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("fear")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 1 june/happy/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("happy")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("happy")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("happy")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("happy")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 1 june/neutral/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("neutral")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("neutral")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("neutral")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("neutral")

    for file in glob.glob("/content/drive/MyDrive/NLP easter 1 june/sad/*.wav"):
        standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(standard_feature)
        y.append("sad")
        wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(wn_feature)
        y.append("sad")
        shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(shift_feature)
        y.append("sad")
        stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(stretch_feature)
        y.append("sad")

    
    #CREAMA-D dataset
    for file in glob.glob("/content/drive/MyDrive/AudioWAV/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("_")[2]]
        if emotion not in observed_emotions:
            continue
        try:
          standard_feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
          x.append(standard_feature)
          y.append(emotion)
          wn_feature=wn_extract_feature(file, mfcc=True, chroma=True, mel=True)
          x.append(wn_feature)
          y.append(emotion)
          shift_feature=shift_extract_feature(file, mfcc=True, chroma=True, mel=True)
          x.append(shift_feature)
          y.append(emotion)
          stretch_feature=stretch_extract_feature(file, mfcc=True, chroma=True, mel=True)
          x.append(stretch_feature)
          y.append(emotion)
        except:
          continue
    
      
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#DataFlair - Split the dataset 16m 5s
x_train,x_test,y_train,y_test=load_data()

In [ ]:
model.save("my_model")

In [ ]:
## Model 1: MLP Classifier

#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(1000,), learning_rate='adaptive', max_iter=5000)

#DataFlair - Train the model
model.fit(x_train,y_train)

#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
def load_interim_data():
    x=[]
    for file in glob.glob("/content/drive/MyDrive/Qualifiers/NLP Final Evaluation Dataset/NLP/*.wav"):
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
    return x

In [ ]:
interim_test = load_interim_data()

In [ ]:
len(interim_test)

In [ ]:
interim_pred=model.predict(interim_test)

In [ ]:
import csv
with open('predictions.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)

    data = list(zip(im2, interim_pred))
    for row in data:
        row = list(row)
        spamwriter.writerow(row)